In [1]:
# Import packages
from datetime import datetime
import pandas as pd
import numpy as np
import os

In [2]:
# Define directory to look at
data_dir = "./Data/"

# Let's have a look
os.listdir(data_dir)

['satisfaction.csv',
 '.DS_Store',
 'stops.csv',
 'train_3605.png',
 'tickets.csv',
 'stations.csv',
 'subscriptions.csv',
 'facilities.csv',
 'travelers.csv',
 'Trips',
 'Overview_NMBS_data.png',
 'incidents.csv']